In [1]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
import pandas as pd
import matplotlib
import pandas_profiling


In [2]:
import os
print(os.getcwd())

C:\Users\kbj80\AppData\Local\Temp\1aebc9e8-e346-4a67-9c79-865643a0a6a0


In [3]:
df = pd.read_excel("D:\Project\Django\Scripts\server_project\\test\owner_project\data\data2018.xlsx")
df.head(5)

,회원번호,성명,주민번호,주소,상태,가입상품,가입일자,최종불입일자,총납입회차,최종불입회차,...,실입금액,기간입금액,기간할인액,잔여금액,조합예수잔액,납입방법,담당자,부서,연체횟수,성별
0,0022A00001,이옥성,590318-1000000,경기도 광주시 퇴촌면 천진암로 530,해약,부금2400000,2008-09-08,2016-12-20,100,100,...,456000,2400000,0,-1944000,0,CMS,더피플라이프,관리부,0,남
1,0072A00001,안성열,581125-1000000,경기도 수원시 팔달구 월드컵로321번길 12-6,행사,특240만원,2007-09-28,2012-10-04,100,100,...,2400000,2400000,0,0,0,CMS,더피플라이프,관리부,0,남
2,0072A00002,배준택,831121-1000000,부산광역시 영도구 와치로 261 신흥동백 8차 다동 507호,보류,특240만원,2007-10-23,2013-05-20,100,68,...,1632000,1632000,0,768000,768000,CMS,더피플라이프,관리부,32,남
3,0072A00003,배민규,821023-1000000,울산광역시 중구 도화골10길 5,행사,특240만원,2007-10-23,2014-12-31,100,100,...,2400000,2400000,0,0,0,CMS,더피플라이프,관리부,0,남
4,0072A00006,최금순,340728-2000000,경기도 과천시 별양로 180 주공8단지 809동 1508호,행사,특240만원,2007-10-31,2016-01-05,100,100,...,2400000,2400000,0,0,0,CMS,더피플라이프,관리부,0,여


(206852,)

In [4]:
df = df[['회원번호','성명','주민번호','주소','상태','가입일자','최종불입일자','총납입회차','최종불입회차','상품금액','총불입액','해약금액','실입금액','기간입금액','기간할인액','잔여금액','조합예수잔액','담당자','부서','연체횟수','성별']]
df2 = df[['회원번호','성명','주민번호','주소','상태','가입일자','최종불입일자','총납입회차','최종불입회차','상품금액','총불입액','해약금액','실입금액','기간입금액','기간할인액','잔여금액','조합예수잔액','담당자','부서','연체횟수','성별']]


In [5]:
def export_adress(adress):
    result = adress[0:2]
    return result

def export_age(idnum):
    age = 120 - (int(idnum[0:2]))
    return age

def transform_status(current_status, flag):
    if current_status == '가입':
        if flag == 0:
            return '만기'
        else:
            return current_status

    elif current_status == '해약':
        if flag == 0:
            return '만기_해약'
        else:
            return current_status

    elif current_status == '미계좌':
        return '해약'

    elif current_status == '보류':
        return '가입'

    elif current_status == '접수':
        return '가입'

    else:
        return current_status
    
def export_price(price):
    if price < 1000000: # 100만원 미만
        return int(0)
    elif price < 2000000: # 200만원 미만
        return 1
    elif price < 3000000:
        return 2
    elif price < 4000000:
        return 3
    elif price < 5000000:
        return 4
    elif price < 6000000:
        return 5
    elif price < 7000000:
        return 6
    elif price < 8000000:
        return 7
    else:
        return 8
print(export_price(1000000))    

def transform_id(name, id_num):
    idnum = id_num[0:6]
    return name + idnum

1


In [6]:
df = df.dropna(how='any')
df.shape

(215094, 21)

In [7]:
df['상품금액']

0         2400000
1         2400000
2         2400000
3         2400000
4         2400000
           ...   
234607    2400000
234609    2400000
234610    2400000
234611    2400000
234612    2400000
Name: 상품금액, Length: 215094, dtype: int64

In [8]:
# 1차 전처리
df['주소'] = df['주소'].apply(lambda x : export_adress(x))
df['나이'] = df['주민번호'].apply(lambda x : export_age(x))
df['상태'] = df.apply(lambda x: transform_status(x['상태'], x['총납입회차'] - x['최종불입회차']), axis=1)
df['상품금액']= df['상품금액'].apply(lambda x : export_price(x))
df['회원이름'] = df.apply(lambda x: transform_id(x['성명'], x['주민번호']), axis=1)


In [9]:
dfs=df[['회원번호','회원이름','주소','상태','가입일자','최종불입일자','총납입회차','최종불입회차','상품금액','총불입액','해약금액','담당자','부서','연체횟수','성별','나이']]

In [19]:
#해약금액 0 인 row 제거
def preprocess_cancle_price(price):
    if price == 0:
        return None
    else:
        return price
    
dfs['해약금액'] = df['해약금액'].apply(lambda x : preprocess_cancle_price(x))

In [21]:
df2 = dfs.dropna(how='any')
df2.shape

(38957, 16)

In [23]:
df2.to_csv("D:\Project\Django\Scripts\server_project\\test\owner_project\해약_2018.csv", index=False)

In [25]:
pr2 = df2.profile_report()
pr2.to_file("해약_report_2018.html")

build report structure: 100%|██████████| 1/1 [00:06<00:00,  6.50s/it]


In [10]:
# 해약금액
def z_score_normalize(value, mean, std):
    normalized = []
    normalized_num = (value - mean) / std
    normalized.append(normalized_num)
    return normalized[0]

In [11]:
mean = np.mean(list(dfs['해약금액']))
std = np.mean(list(dfs['해약금액']))
dfs['해약금액'] = dfs['해약금액'].apply(lambda x : z_score_normalize(x,mean,std))

In [12]:
mean = np.mean(list(dfs['연체횟수']))
std = np.mean(list(dfs['연체횟수']))
dfs['연체횟수'] = dfs['연체횟수'].apply(lambda x : z_score_normalize(x,mean,std))

In [13]:
mean = np.mean(list(dfs['나이']))
std = np.mean(list(dfs['나이']))
dfs['나이'] = dfs['나이'].apply(lambda x : z_score_normalize(x,mean,std))

In [14]:
mean = np.mean(list(dfs['총불입액']))
std = np.mean(list(dfs['총불입액']))
dfs['총불입액'] = dfs['총불입액'].apply(lambda x : z_score_normalize(x,mean,std))

In [25]:
dfs['상품금액'] = df2['상품금액']
dfs

,회원번호,회원이름,주소,상품금액,총불입액,해약금액,담당자,연체횟수,성별,나이,진행률,상태
0,0022A00001,이옥성590318,경기,2400000,3.043359,18.730409,더피플라이프,-1.000000,0,0.097929,1.000000,0
1,0072A00001,안성열581125,경기,2400000,3.043359,-1.000000,더피플라이프,-1.000000,0,0.115928,1.000000,2
2,0072A00002,배준택831121,부산,2400000,1.749484,-1.000000,더피플라이프,0.838945,0,-0.334043,0.680000,3
3,0072A00003,배민규821023,울산,2400000,3.043359,-1.000000,더피플라이프,-1.000000,0,-0.316044,1.000000,2
4,0072A00006,최금순340728,경기,2400000,3.043359,-1.000000,더피플라이프,-1.000000,1,0.547900,1.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...
234607,U022A22155,조길찬761012,부산,2400000,-0.878699,-1.000000,더피플라이프,4.574302,0,-0.208051,0.030000,1
234609,U244A00803,배상호820807,부산,2400000,1.897741,11.585241,더피플라이프,-0.023060,0,-0.316044,0.716667,1
234610,U244A00804,배규태830402,부산,2400000,3.043359,-1.000000,더피플라이프,-1.000000,0,-0.334043,1.000000,4
234611,U244A00805,김군자490809,부산,2400000,-0.932611,-1.000000,더피플라이프,2.390555,1,0.277917,0.016667,1


In [26]:
mean = np.mean(list(dfs['상품금액']))
std = np.mean(list(dfs['상품금액']))
dfs['상품금액'] = dfs['상품금액'].apply(lambda x : z_score_normalize(x,mean,std))

In [16]:
def process_rate(total, last):
    rate = int(last) / int(total)
    return rate


In [17]:
dfs['진행률'] = dfs.apply(lambda x: process_rate(x['총납입회차'] , x['최종불입회차']), axis=1)


In [18]:
dfs=dfs[['회원번호','회원이름','주소','상태','상품금액','총불입액','해약금액','담당자','연체횟수','성별','나이','진행률']]
dfs

,회원번호,회원이름,주소,상태,상품금액,총불입액,해약금액,담당자,연체횟수,성별,나이,진행률
0,0022A00001,이옥성590318,경기,만기_해약,2,3.043359,18.730409,더피플라이프,-1.000000,남,0.097929,1.000000
1,0072A00001,안성열581125,경기,행사,2,3.043359,-1.000000,더피플라이프,-1.000000,남,0.115928,1.000000
2,0072A00002,배준택831121,부산,가입,2,1.749484,-1.000000,더피플라이프,0.838945,남,-0.334043,0.680000
3,0072A00003,배민규821023,울산,행사,2,3.043359,-1.000000,더피플라이프,-1.000000,남,-0.316044,1.000000
4,0072A00006,최금순340728,경기,행사,2,3.043359,-1.000000,더피플라이프,-1.000000,여,0.547900,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
234607,U022A22155,조길찬761012,부산,해약,2,-0.878699,-1.000000,더피플라이프,4.574302,남,-0.208051,0.030000
234609,U244A00803,배상호820807,부산,해약,2,1.897741,11.585241,더피플라이프,-0.023060,남,-0.316044,0.716667
234610,U244A00804,배규태830402,부산,만기,2,3.043359,-1.000000,더피플라이프,-1.000000,남,-0.334043,1.000000
234611,U244A00805,김군자490809,부산,해약,2,-0.932611,-1.000000,더피플라이프,2.390555,여,0.277917,0.016667


In [19]:
def preprocess_sex(sex):
    if sex =='남':
        return 0
    else:
        return 1

In [20]:
dfs['성별'] = dfs['성별'].apply(lambda x : preprocess_sex(x))

In [21]:
def labeling(status):
    if status == '만기_해약':
        return 0
    elif status == '해약':
        return 1
    elif status == '행사':
        return 2
    elif status == '가입':
        return 3
    else: # 만기
        return 4

In [22]:
dfs['상태'] = dfs['상태'].apply(lambda x : labeling(x))

In [27]:
dfs=dfs[['회원번호','회원이름','주소','상품금액','총불입액','해약금액','담당자','연체횟수','성별','나이','진행률','상태']]
dfs

,회원번호,회원이름,주소,상품금액,총불입액,해약금액,담당자,연체횟수,성별,나이,진행률,상태
0,0022A00001,이옥성590318,경기,-0.299494,3.043359,18.730409,더피플라이프,-1.000000,0,0.097929,1.000000,0
1,0072A00001,안성열581125,경기,-0.299494,3.043359,-1.000000,더피플라이프,-1.000000,0,0.115928,1.000000,2
2,0072A00002,배준택831121,부산,-0.299494,1.749484,-1.000000,더피플라이프,0.838945,0,-0.334043,0.680000,3
3,0072A00003,배민규821023,울산,-0.299494,3.043359,-1.000000,더피플라이프,-1.000000,0,-0.316044,1.000000,2
4,0072A00006,최금순340728,경기,-0.299494,3.043359,-1.000000,더피플라이프,-1.000000,1,0.547900,1.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...
234607,U022A22155,조길찬761012,부산,-0.299494,-0.878699,-1.000000,더피플라이프,4.574302,0,-0.208051,0.030000,1
234609,U244A00803,배상호820807,부산,-0.299494,1.897741,11.585241,더피플라이프,-0.023060,0,-0.316044,0.716667,1
234610,U244A00804,배규태830402,부산,-0.299494,3.043359,-1.000000,더피플라이프,-1.000000,0,-0.334043,1.000000,4
234611,U244A00805,김군자490809,부산,-0.299494,-0.932611,-1.000000,더피플라이프,2.390555,1,0.277917,0.016667,1


In [28]:
pr = dfs.profile_report()
pr.to_file("normalized_report.html")

build report structure: 100%|██████████| 1/1 [00:06<00:00,  6.53s/it]


In [29]:
dfs.to_csv('normalization.csv')